In [1]:
import os
import sys

import litellm
import mlflow
from dotenv import load_dotenv
from mlflow.genai import scorer
from mlflow.genai.scorers import Correctness, Guidelines
from openai import OpenAI

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [6]:
load_dotenv()

if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

print('OPENAI_API_KEY=' + os.getenv('OPENAI_API_KEY')[:20] + '...')
print('GEMINI_API_KEY=' + os.getenv('GEMINI_API_KEY')[:5] + '...')


OPENAI_API_KEY=sk-proj-ZhcCqw0r70dP...
GEMINI_API_KEY=AIzaS...


In [7]:
# This will create a new experiment called "GenAI Evaluation Quickstart" and set it as active
mlflow.set_experiment("GenAI Evaluation Quickstart")


<Experiment: artifact_location='mlflow-artifacts:/7', creation_time=1758570485868, experiment_id='7', last_update_time=1758570485868, lifecycle_stage='active', name='GenAI Evaluation Quickstart', tags={'mlflow.experimentKind': 'genai_development'}>

In [13]:
def qa_predict_fn_openai_direct(question: str) -> str:
    """Simple Q&A prediction function using OpenAI"""

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Answer questions concisely.",
            },
            {"role": "user", "content": question},
        ],
    )
    return response.choices[0].message.content


def qa_predict_fn_gemini_flash_litellm(question: str) -> str:
    response = litellm.completion(
        model="gemini/gemini-2.5-flash-preview-05-20",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Answer questions concisely.",
            },
            {"role": "user", "content": question},
        ],
    )
    return response.choices[0].message.content


def build_qa_predict_fn(model: str) -> str:
    def predict_fn(question):
        litellm.completion(
            model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant. Answer questions concisely.",
                },
                {"role": "user", "content": question},
            ],
        ).choices[0].message.content

    return predict_fn


In [9]:
# Define a simple Q&A dataset with questions and expected answers
eval_dataset = [
    {
        "inputs": {"question": "What is the capital of France?"},
        "expectations": {"expected_response": "Paris"},
    },
    {
        "inputs": {"question": "Who was the first person to build an airplane?"},
        "expectations": {"expected_response": "Wright Brothers"},
    },
    {
        "inputs": {"question": "Who wrote Romeo and Juliet?"},
        "expectations": {"expected_response": "William Shakespeare"},
    },
]


In [10]:
@scorer
def is_concise(outputs: str) -> bool:
    """Evaluate if the answer is concise (less than 5 words)"""
    return len(outputs.split()) <= 5


scorers = [
    Correctness(),
    Guidelines(name="is_english", guidelines="The answer must be in English"),
    is_concise,
]


In [14]:
results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=build_qa_predict_fn("gpt-4o-mini"),
    scorers=scorers,
)

2025/10/20 21:19:28 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
Evaluating: 100%|██████████| 3/3 [Elapsed: 00:04, Remaining: 00:00] 
2025/10/20 21:19:33 ERROR mlflow.genai.scorers.aggregation: Invalid assessment value for is_concise: None
2025/10/20 21:19:33 ERROR mlflow.genai.scorers.aggregation: Invalid assessment value for is_concise: None
2025/10/20 21:19:33 ERROR mlflow.genai.scorers.aggregation: Invalid assessment value for is_concise: None


In [12]:
results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=build_qa_predict_fn("gemini/gemini-2.5-flash-preview-09-2025"),
    scorers=scorers,
)

2025/10/20 21:15:28 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
Evaluating: 100%|██████████| 3/3 [Elapsed: 00:03, Remaining: 00:00] 
